<a href="https://colab.research.google.com/github/JoaoAlexandreFerreira/PINNs/blob/main/PINN_Inversa_Navier_Stokes_pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.autograd as autograd         #AD, diferenciação automática
from torch import Tensor
import torch.nn as nn                     #rede neural
import torch.optim as optim               #otimizadores
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker
from sklearn.model_selection import train_test_split
import numpy as np
import time
import scipy.io

#Padronizando o tipo dos valores
torch.set_default_dtype(torch.float)
torch.manual_seed(1234) #Uma semente para os valores, evitando a geração aleatória
np.random.seed(1234)

#Esse aqui não sei bem, mas sempre usam
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(device)

if device == 'cuda':
    print(torch.cuda.get_device_name())

cpu


In [2]:
class RNA(nn.Module):
    def __init__(self,camadas):
        super().__init__()

        #Função de ativação
        self.ativacao = nn.Tanh()

        #Criando a rede neural
        self.linears = nn.ModuleList([nn.Linear(camadas[i], camadas[i+1]) for i in range(len(camadas)-1)])

        #inicialização normal
        for i in range(len(camadas)-1):
            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)


    #propagação direta
    def forward(self,x):

        if torch.is_tensor(x) != True:
            x = torch.from_numpy(x)

        u_b = torch.from_numpy(ub).float().to(device)
        l_b = torch.from_numpy(lb).float().to(device)

        #Normalizando os dados
        x = (x - l_b)/(u_b - l_b)

        a = x.float()

        for i in range(len(camadas)-2):

            z = self.linears[i](a)
            a = self.ativacao(z)

        a = self.linears[-1](a)
        return a

In [3]:
#Inicialização dos parâmetros
lambda_1 = 0.0
lambda_2 = 0.0

In [4]:
#Classe da pinn para navier stokes
class PINN_NS():
  def __init__(self, camadas, x, y, t, u, v, p):

    self.x = torch.tensor(x, dtype=torch.float32, requires_grad=True).to(device)
    self.y = torch.tensor(y, dtype=torch.float32, requires_grad=True).to(device)
    self.t = torch.tensor(t, dtype=torch.float32, requires_grad=True).to(device)

    self.u = torch.tensor(u, dtype=torch.float32, requires_grad=True).to(device)
    self.v = torch.tensor(v, dtype=torch.float32, requires_grad=True).to(device)
    self.p = torch.tensor(p, dtype=torch.float32, requires_grad=True).to(device)

    self.mse = nn.MSELoss(reduction = 'mean') #Mean Squared Error, tirado do próprio pytorch
    self.itr = 0 #Contador de iteração

    self.lambda_1 = torch.tensor([lambda_1], requires_grad=True).float().to(device)
    self.lambda_2 = torch.tensor([lambda_2], requires_grad=True).float().to(device)

    self.lambda_1 = nn.Parameter(self.lambda_1) #Colocando os lambdas como parametro do pytorch (https://stackoverflow.com/questions/50935345/understanding-torch-nn-parameter)
    self.lambda_2 = nn.Parameter(self.lambda_2)

    self.modelo = RNA(camadas).to(device) #Instanciando a rede neural
    self.modelo.register_parameter('lambda_1', self.lambda_1) #Agora está setando os lambdas como parametro da rede neural (https://pytorch.org/docs/stable/generated/torch.nn.Module.html)
    self.modelo.register_parameter('lambda_2', self.lambda_2)

    self.otimizador = optim.LBFGS(self.modelo.parameters(), lr=0.01, max_iter=200000, max_eval=50000,history_size=50,
                                  tolerance_grad=1e-05, tolerance_change=0.5 * np.finfo(float).eps,line_search_fn="strong_wolfe")

  def EDP(self):
    lambda_1 = self.lambda_1
    lambda_2 = self.lambda_2

    psi_p = self.modelo(torch.hstack((self.x, self.y, self.t)))
    psi, p = psi_p[:, 0:1], psi_p[:, 1:2]

    u = autograd.grad(psi, self.y, torch.ones_like(psi), create_graph=True)[0]
    v = -1.*autograd.grad(psi, self.x, torch.ones_like(psi), create_graph=True)[0]

    u_x = autograd.grad(u, self.x, torch.ones_like(u), create_graph = True)[0]
    u_xx = autograd.grad(u_x, self.x, torch.ones_like(u_x), create_graph = True)[0]

    u_y = autograd.grad(u, self.y, torch.ones_like(u), create_graph = True)[0]
    u_yy = autograd.grad(u_y, self.y, torch.ones_like(u_y), create_graph = True)[0]

    u_t = autograd.grad(u, self.t, torch.ones_like(u), create_graph = True)[0]

    v_x = autograd.grad(v, self.x, torch.ones_like(v), create_graph = True)[0]
    v_xx = autograd.grad(v_x, self.x, torch.ones_like(v_x), create_graph = True)[0]

    v_y = autograd.grad(v, self.y, torch.ones_like(v), create_graph = True)[0]
    v_yy = autograd.grad(v_y, self.y, torch.ones_like(v_y), create_graph = True)[0]

    v_t = autograd.grad(v, self.t, torch.ones_like(v), create_graph = True)[0]

    p_x = autograd.grad(p, self.x, torch.ones_like(p), create_graph = True)[0]
    p_y = autograd.grad(p, self.y, torch.ones_like(p), create_graph = True)[0]


    f = u_t + (lambda_1)*(u*u_x + v*u_y) + p_x - (lambda_2)*(u_xx + u_yy)
    g = v_t + (lambda_1)*(u*v_x + v*v_y) + p_y - (lambda_2)*(v_xx + v_yy)

    return u, v, p, f, g

  def erro(self):

    u_pred, v_pred, p_pred, f, g = self.EDP()

    #Erro dos dados
    erro_u = self.mse(u_pred, self.u)
    erro_v = self.mse(v_pred, self.v)

    #Erro da edp
    erro_f = self.mse(f, torch.zeros_like(f))
    erro_g = self.mse(g, torch.zeros_like(g))

    #Erro total
    erro_total = erro_u + erro_v + erro_f + erro_g

    return erro_total

  def treino_otimizador(self):

    self.otimizador.zero_grad()

    erro_total = self.erro()
    erro_total.backward()

    self.itr +=1

    if self.itr % 10 == 0:
      print('Iteração: {:}, Custo: {:0.6f}, 𝜆_PINN = {:.5f},  {:.5f}'.format(self.itr, erro_total, self.lambda_1.item(), self.lambda_2.item()))

    return erro_total

  def treino(self):
    self.otimizador.step(self.treino_otimizador)

###Aqui é puramente o código do maziar para tratamento dos dados

In [5]:
N_train = 5000

data = scipy.io.loadmat('cylinder_wake.mat')

U_star = data['U_star']  # N x 2 x T
P_star = data['p_star']  # N x T
t_star = data['t']  # T x 1
X_star = data['X_star']  # N x 2

N = X_star.shape[0]
T = t_star.shape[0]

x_test = X_star[:, 0:1]
y_test = X_star[:, 1:2]
p_test = P_star[:, 0:1]
u_test = U_star[:, 0:1, 0]
t_test = np.ones((x_test.shape[0], x_test.shape[1]))


# Rearrange Data
XX = np.tile(X_star[:, 0:1], (1, T))  # N x T
YY = np.tile(X_star[:, 1:2], (1, T))  # N x T
TT = np.tile(t_star, (1, N)).T  # N x T

UU = U_star[:, 0, :]  # N x T
VV = U_star[:, 1, :]  # N x T
PP = P_star  # N x T

x = XX.flatten()[:, None]  # NT x 1
y = YY.flatten()[:, None]  # NT x 1
t = TT.flatten()[:, None]  # NT x 1

u = UU.flatten()[:, None]  # NT x 1
v = VV.flatten()[:, None]  # NT x 1
p = PP.flatten()[:, None]  # NT x 1

# Training Data
idx = np.random.choice(N * T, N_train, replace=False)
x_train = x[idx, :]
y_train = y[idx, :]
t_train = t[idx, :]
u_train = u[idx, :]
v_train = v[idx, :]
p_train = p[idx, :]

lb = np.hstack((np.min(x_train), np.min(y_train), np.min(t_train)))
ub = np.hstack((np.max(x_train), np.max(y_train), np.max(t_train)))

In [6]:
itr=20000
lr=0.1
camadas = np.array([3,20,20,20,20,20,20,20,20,2]) #2 nós de entradas, 8 camadas ocultas contendo 20 nós por camadas e uma saída
N_u = 100 #Número total de pontos de dados para 'u'
N_f = 25000 # Número total de pontos de colocação
nu = 0.01/np.pi #Coeficiente de difusão

In [ ]:
pinn_ns = PINN_NS(camadas, x_train, y_train, t_train, u_train, v_train, p_train)

pinn_ns.treino()

Iteração: 10, Custo: 0.298933, 𝜆_PINN = 0.00000,  -0.00005
Iteração: 20, Custo: 0.232909, 𝜆_PINN = 0.00020,  0.00029
Iteração: 30, Custo: 0.162005, 𝜆_PINN = 0.00050,  0.00187
Iteração: 40, Custo: 0.144140, 𝜆_PINN = 0.00070,  0.00342
Iteração: 50, Custo: 0.133499, 𝜆_PINN = 0.00081,  0.00411
Iteração: 60, Custo: 0.127399, 𝜆_PINN = 0.00106,  0.00256
Iteração: 70, Custo: 0.126264, 𝜆_PINN = 0.00128,  0.00069
Iteração: 80, Custo: 0.123317, 𝜆_PINN = 0.00172,  -0.00520
Iteração: 90, Custo: 0.120544, 𝜆_PINN = 0.00209,  -0.00823
Iteração: 100, Custo: 0.116374, 𝜆_PINN = 0.00271,  -0.00461
Iteração: 110, Custo: 0.110500, 𝜆_PINN = 0.00217,  0.02514
Iteração: 120, Custo: 0.106877, 𝜆_PINN = 0.00270,  0.01490
Iteração: 130, Custo: 0.104790, 𝜆_PINN = 0.00409,  -0.00840
Iteração: 140, Custo: 0.103507, 𝜆_PINN = 0.00448,  -0.00691
Iteração: 150, Custo: 0.100778, 𝜆_PINN = 0.00516,  0.00216
Iteração: 160, Custo: 0.100221, 𝜆_PINN = 0.00506,  -0.00221
Iteração: 170, Custo: 0.099468, 𝜆_PINN = 0.00489,  -0.0036